# Game observations
- Player/Opponent fleets
- Player/Opponent shipyards
- Kore (animated 3D plot)

In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import bz2
import pickle
import _pickle as cPickle
import matplotlib.animation as animation
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400) 

In [ ]:
def compressed_pickle(filename, data):
    with bz2.BZ2File(filename + '.pbz2', 'w') as f: 
         cPickle.dump(data, f)

def decompress_pickle(filename):
    data = bz2.BZ2File(filename, 'rb')
    data = cPickle.load(data)
    return data            

In [ ]:
from kaggle_environments import utils
from kaggle_environments.helpers import Point, Direction
from kaggle_environments.envs.kore_fleets.helpers import Board, ShipyardAction

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

In [ ]:
full_game = env.run(["random", "random"])
env.render(mode="ipython", width=900, height=800) # , width=1000, height=800

In [ ]:
GRID_SIZE = 21

# Kore available

In [ ]:
def get_kore(full_game_):
#     kore_grid_ = np.zeros((len(full_game_), GRID_SIZE, GRID_SIZE), dtype=np.float32)
#     env = make("kore_fleets", configuration={"size": GRID_SIZE,"startingKore": 2750})
#     for turn in range(len(full_game_)):        
#         board = Board(full_game_[turn][0]["observation"], env.configuration)
#         for y in range(GRID_SIZE):
#             for x in range(GRID_SIZE):
#                 kore_grid_[turn, y, x] = board.cells.get((x, y)).kore    
    kore_grid_ = np.stack([np.flip(np.array(full_game_[turn][0]["observation"]["kore"], dtype=np.float32).reshape(GRID_SIZE, GRID_SIZE), axis=0) for turn in range(len(full_game_))])
    return kore_grid_

kore_grid = get_kore(full_game)
kore_grid.shape

# Player/Opponent fleets

In [ ]:
def get_fleets(player_fleets_):
    player_fleets_grid = np.zeros((len(full_game), GRID_SIZE, GRID_SIZE), dtype=np.int32)
    for turn, player_fleets_turn in enumerate(player_fleets_):
        for (fleet_id, [fleet_index, fleet_kore, ship_count, direction, flight_plan]) in player_fleets_turn.items():
            (x, y) = Point.from_index(fleet_index, GRID_SIZE)
            fleet_direction = Direction.from_index(direction)        
            player_fleets_grid[turn, y, x] = ship_count
    return player_fleets_grid

# Player fleets
player_fleets_grid = get_fleets([turn[0]["observation"]["players"][0][2] for turn in full_game])

# Opponent fleets
opponent_fleets_grid = get_fleets([turn[0]["observation"]["players"][1][2] for turn in full_game])

# Player/Opponent shipyards

In [ ]:
def get_name_player(actions_):
    player = {}
    for action_ in actions_:
        names_player = list(action_.keys())
        for name_player in names_player:
            if player.get(name_player) is None:
                player[name_player] = 1
            else:
                player[name_player] = player[name_player] + 1
    return player

def get_player_shipyard(player_shipyards_):
    player_shipyards_dict_ = {}
    player_shipyards_grid = np.zeros((len(player_shipyards_), GRID_SIZE, GRID_SIZE), dtype=np.int32)
    # For each shipyard
    for player_init in list(get_name_player(player_shipyards_).keys()):
        shipyard_pd = []
        # For each turn, get shipyard
        for turn, player_shipyard_ in enumerate(player_shipyards_, 0):
            shipyard_ = player_shipyard_.get(player_init)
            # If shipyard available in the current turn
            if shipyard_ is not None:
                shipyard_index, ship_count, turns_controlled = shipyard_
                shipyard_pd.append((player_init, turn, shipyard_index, ship_count, turns_controlled))
        shipyards_pd = pd.DataFrame(shipyard_pd, columns=["name_player", "turn", "location", "ship_count", "turns_controlled"])
        player_shipyards_dict_[player_init] = shipyards_pd
        for turn in range(len(player_shipyards_)):
            turn_pd = shipyards_pd[shipyards_pd["turn"] == turn]
            if len(turn_pd) > 0:
                shipyard_index, shipyard_ships = turn_pd[["location", "ship_count"]].values[0]
                (x, y) = Point.from_index(shipyard_index, GRID_SIZE)
                player_shipyards_grid[turn, y, x] = shipyard_ships + 1 # +1 to make it exist
    return player_shipyards_dict_, player_shipyards_grid

player_shipyards_dict, player_shipyards_grid = get_player_shipyard([turn[0]["observation"]["players"][0][1] for turn in full_game])
opponent_shipyards_dict, opponent_shipyards_grid = get_player_shipyard([turn[0]["observation"]["players"][1][1] for turn in full_game])

# Save observations

In [ ]:
game_id = "123456"

game_observations = {
    'kore': kore_grid, # Kore per cell, np.float32
    'player_fleets': player_fleets_grid, # Ships per cell, np.int32
    'opponent_fleets': opponent_fleets_grid, # Ships per cell, np.int32
    'player_shipyards': player_shipyards_grid, # Shipyards ships+1 per cell, np.int32
    'opponent_shipyards': opponent_shipyards_grid, # Shipyards ships+1 per cell, np.int32
}

compressed_pickle(game_id, game_observations)

In [ ]:
turn_total = player_fleets_grid.shape[0]
turn_init = 19
for turn in [turn_init, turn_total-50]:
    fig, ax = plt.subplots(3,2, figsize=(14, 20))
    d = sns.heatmap(player_fleets_grid[turn], annot=True, fmt=".0f", ax=ax[0,0], cbar=False)
    d = sns.heatmap(opponent_fleets_grid[turn], annot=True, fmt=".0f", ax=ax[0,1], cbar=False)
    d = sns.heatmap(player_shipyards_grid[turn], annot=True, fmt=".0f", ax=ax[1,0], cbar=False)
    d = sns.heatmap(opponent_shipyards_grid[turn], annot=True, fmt=".0f", ax=ax[1,1], cbar=False)
    
    d = sns.heatmap(kore_grid[turn], annot=True, fmt=".0f", ax=ax[2,0], cbar=False)
    
    d = ax[0,0].invert_yaxis()
    d = ax[0,0].set_title("Player fleets - Turn %d / %d" % (turn+1, turn_total))
    d = ax[0,1].invert_yaxis()
    d = ax[0,1].set_title("Opponent fleets - Turn %d / %d" % (turn+1, turn_total))    
    d = ax[1,0].invert_yaxis()
    d = ax[1,0].set_title("Player shipyards - Turn %d / %d" % (turn+1, turn_total))
    d = ax[1,1].invert_yaxis()
    d = ax[1,1].set_title("Opponent shipyards - Turn %d / %d" % (turn+1, turn_total))
    
    d = ax[2,0].invert_yaxis()
    d = ax[2,0].set_title("Kore available - Turn %d / %d" % (turn+1, turn_total))    
    
    fig.tight_layout()

# Total Kore

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(22, 5))

kore_pd = pd.DataFrame([x for x in range(len(full_game))], columns=["turn"])
kore_pd["kore_total"] = np.sum(kore_grid, axis=(1,2))
kore_pd["kore_mean"] = np.mean(kore_grid, axis=(1,2))
kore_pd["kore_max"] = np.max(kore_grid, axis=(1,2))
kore_pd["kore_min"] = np.min(kore_grid, axis=(1,2))                     
d = kore_pd.plot(kind="line", x="turn", y=["kore_total"], ax=ax[0], grid=True)
d = kore_pd.plot(kind="line", x="turn", y=["kore_mean", "kore_max", "kore_min"], ax=ax[1], grid=True)

fig.tight_layout()

In [ ]:
kore_pd.head()

# Reward

In [ ]:
scores_pd = []
for turn in range(len(full_game)):
    player_score = full_game[turn][0]["reward"]
    opponent_score = full_game[turn][1]["reward"]
    scores_pd.append((turn, player_score, opponent_score))
scores_pd = pd.DataFrame(scores_pd, columns=["turn", "player_reward", "opponent_reward"])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(22, 5))
d = scores_pd.plot(kind="line", x="turn", ax=ax, grid=True)

# Kore

In [ ]:
x = np.linspace(0, 20, 21)
y = np.linspace(0, 20, 21)
X, Y = np.meshgrid(x, y)
    
fig, ax = plt.subplots(1,1, figsize=(18, 13))
ax = plt.axes(projection='3d')
txt_title = ax.set_title('')
ax_view = ax.view_init(75, 40)
d = ax.set_zlim(0, kore_pd["kore_max"].max())

surface = ax.plot_surface(X, Y, kore_grid[0], cmap='magma', rstride=1, cstride=1) # , edgecolor='none'
d = txt_title.set_text('Turn = {:d}'.format(0))

fig.tight_layout()

def animate(i):
    # ax.clear()
    ax.set_zlim(0, kore_pd["kore_max"].max())
    Z = kore_grid[i]
    ax.plot_surface(X, Y, Z, cmap='magma', rstride=1, cstride=1)
    txt_title.set_text('Turn = {:d}'.format(i))
    return surface,

# Kore animation

In [ ]:
anim = animation.FuncAnimation(fig, animate, frames=turn_total, blit=True, interval=500, repeat=False)

In [ ]:
anim.save('kore.gif', writer='imagemagick', fps=30)
from IPython.display import HTML, Image
Image(url='kore.gif')